If for some reason, some environmental variables are missing and `DBSingleton` won't start, run the following to set them. 
Need the latest `DOSlib` and `PositionerIndex`

In [34]:
import os
envvars = {'DOS_CONSTANTS_DB_HOST': 'desi-db',
           'DOS_CONSTANTS_DB_PORT': '5442',
           'DOS_CONSTANTS_DB_USER': 'desi_reader',
           'DOS_CONSTANTS_DB_PASSWORD': 'reader',
           'DOS_CONSTANTS_DB_NAME': 'desi_dev',
           'DOS_POSITIONERINDEXTABLE': '/software/products/PositionerIndexTable-trunk/index_files/desi_positioner_indexes_20191108.csv'
}
for key, val in envvars.items():
    if os.getenv(key) is None:
        os.environ[key] = val

In [80]:
petal_id = 4  # specify petal id here
db = DBSingleton(petal_id)

In [87]:
import pandas as pd
df = pd.DataFrame.from_dict(db.posids, orient='index')  # create df for positioners
# select only those on PETAL petal_id, note thatsome of db's attributes include all petals while some don't
df = df[df['PETAL_ID'] == petal_id].reset_index(drop=True)
# make pos constants into a df for merging
df_const = pd.DataFrame.from_dict(db.pos_constants['fiber_positioner'], orient='index').reset_index().rename(columns={'index': 'POS_ID'})
# make pos calib values into a df for merging
df_calib = pd.DataFrame.from_dict(db.pos_calib, orient='index').reset_index(drop=True)

In [94]:
df = df.merge(df_const, on='POS_ID').merge(df_calib, on='POS_ID')  # merge everything into one df
df  # now has all positioner constants and calibration values

,POS_ID,BUS_ID_x,CAN_ID,PETAL_ID_x,DEVICE_LOC_x,BACKLASH,CURR_HOLD,CURR_CREEP,MOTOR_ID_P,MOTOR_ID_T,...,OFFSET_T,OFFSET_P,PHYSICAL_RANGE_T,PHYSICAL_RANGE_P,GEAR_CALIB_T,GEAR_CALIB_P,TOTAL_LIMIT_SEEKS_T,TOTAL_LIMIT_SEEKS_P,LAST_PRIMARY_HARDSTOP_DIR_T,LAST_PRIMARY_HARDSTOP_DIR_P
0,M02691,16,2691,4,486,3.0,0,100,0,1,...,-103.649792,-1.540028,392.554285,188.730383,1.0,1.0,144.0,147.0,-1.0,1.0
1,M02780,16,2780,4,518,3.0,0,100,0,1,...,153.645055,-1.643213,391.481508,191.614773,1.0,1.0,146.0,149.0,-1.0,1.0
2,M02791,16,2791,4,485,3.0,0,100,0,1,...,-57.975212,-4.882534,391.107134,190.998690,1.0,1.0,146.0,149.0,-1.0,1.0
3,M02895,16,2895,4,470,3.0,0,100,0,1,...,-31.483628,2.125624,391.819739,193.489799,1.0,1.0,152.0,156.0,-1.0,1.0
4,M02910,10,2910,4,19,3.0,0,100,0,1,...,-126.376320,-1.324797,391.279653,191.508107,1.0,1.0,180.0,188.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,M08194,13,8194,4,115,3.0,0,100,0,1,...,43.656991,-2.301577,393.299495,195.210142,1.0,1.0,144.0,147.0,-1.0,1.0
498,M08195,13,8195,4,83,3.0,0,100,0,1,...,80.580388,-2.939332,393.318582,192.170463,1.0,1.0,144.0,147.0,-1.0,1.0
499,M08196,13,8196,4,128,3.0,0,100,0,1,...,91.896219,-0.466589,394.952731,188.537514,1.0,1.0,144.0,147.0,-1.0,1.0
500,M02956,17,2956,4,501,3.0,0,100,0,1,...,0.000000,0.000000,389.511676,190.802074,1.0,1.0,102.0,105.0,-1.0,1.0


get petal alignment data

In [105]:
from DOSlib.constants import ConstantsDB
petal_alignments = ConstantsDB().get_constants(group='focal_plane_metrology', snapshot='DESI', tag='CURRENT')['focal_plane_metrology']
for petal_loc, val in petal_alignments.items():
    if val['petal_id'] == petal_id:
        petal_alignment = val
        break

Default-DEFAULT [INFO   ] 2019-11-11 23:52:41Z Fetching tag CURRENT snapshot DESI version 80
Default-DEFAULT [INFO   ] 2019-11-11 23:52:41Z Fetching snapshot DESI version 80 group focal_plane_metrology


perform transformations

In [117]:
# again, take care of missing env var in this NB
import sys
sys.path.append('/software/products/plate_control-trunk/petal')
os.environ['POSITIONER_LOGS_PATH'] = '/data/focalplane/logs'
os.environ['FP_SETTINGS_PATH'] = '/data/focalplane/fp_settings'

In [120]:
from petaltransforms import PetalTransforms
trans = PetalTransforms(Tx=petal_alignment['petal_offset_x'],
                        Ty=petal_alignment['petal_offset_y'],
                        Tz=petal_alignment['petal_offset_z'],
                        alpha=petal_alignment['petal_rot_1'],
                        beta=petal_alignment['petal_rot_2'],
                        gamma=petal_alignment['petal_rot_3'])
offsets_obsXY = trans.flatXY_to_obsXY(df[['OFFSET_X', 'OFFSET_Y']].T.values)
df['OFFSETS_obsX'], df['OFFSETS_obsY'] = offsets_obsXY[0, :], offsets_obsXY[1, :]